# Introduction à Langchain 



### Dépendances et clé d'API

In [3]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")



## Vanilla via api OpenAI

Sortez les rames :)

In [13]:
from openai import OpenAI
from openai.types.responses import ResponseFunctionToolCall


client = OpenAI()


def initial_llm_request(user_message: str):
    """
    Make the initial request to the LLM with the user's message.
    """
    tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]
    return client.responses.create(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": user_message}],
        tools=tools
    )

def get_weather(location: str) -> str:
    """
    Simulate a weather API call.
    In a real-world scenario, you would replace this with an actual API call to a weather service.
    """
    # Simulated response
    return f"The current temperature in {location} is 20°C with clear skies."



def handle_tool_call(response):
    """
    Handle the tool call response from the LLM.
    If the response contains a tool call, execute it and return the result.
    """
    if isinstance(response.output[0], ResponseFunctionToolCall):
        tool_call = response.output[0]
        if tool_call.name == "get_weather":
            import json
            tool_args = json.loads(tool_call.arguments)
            location = tool_args.get("location", "")
            weather = get_weather(location)
            return weather
    return None


def final_llm_call(user_message: str, weather_info: str):
    """
    Make a final call to the LLM with the weather information.
    """
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": user_message}, {"role": "developer", "content": weather_info}],
    )

    return response.output_text



user_message = "What is the weather like in Paris today?"
initial_response = initial_llm_request(user_message)
weather_info = handle_tool_call(initial_response)
if weather_info:
    final_response = final_llm_call(user_message, weather_info)
    print(final_response)
else:
    print("No tool call was made by the LLM.")


Today in Paris, the weather is clear with a temperature of 20°C. It sounds like a lovely day to explore the city!


## Langchain Tool 

In [ ]:
%pip install langchain-tavily

ERROR: unknown command "inqtall" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    api_key="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",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [6]:
tool.invoke({"query": "What happened at the last Roland Garros?"})

{'query': 'What happened at the last Roland Garros?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'At final Roland Garros, Danielle Collins suffers shock exit to Olga ...',
   'url': 'https://www.tennis.com/news/articles/danielle-collins-exit-final-roland-garros-olga-danilovic-retirement-novak-djokovic',
   'content': 'Danielle Collins bowed out of her final Roland Garros in the second round, losing to Olga Danilovic, 6-7 (3), 7-5, 6-4. Collins, who plans to retire at the end of the 2024 season, arrived in Paris',
   'score': 0.5163278,
   'raw_content': None},
  {'title': "Novak Djokovic: Roland Garros loss 'could have been the last match I ...",
   'url': 'https://www.atptour.com/en/news/djokovic-roland-garros-2025-sf-reaction',
   'content': 'The 38-year-old spoke candidly in his post-match press conference, reflecting on the uncertainty of the future following his semi-final loss to top seed Jannik Sinner at Roland Garros. "This could have b

## Tool et Agent

### Web search

In [7]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "Who won Roland Garros 2025? Include only wikipedia sources."

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Who won Roland Garros 2025? Include only wikipedia sources.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_H9xhV9TS1kJmDyx0cLCbuDW8)
 Call ID: call_H9xhV9TS1kJmDyx0cLCbuDW8
  Args:
    query: Roland Garros 2025 winner site:wikipedia.org
    include_domains: ['wikipedia.org']
================================= Tool Message =================================
Name: tavily_search

{"query": "Roland Garros 2025 winner site:wikipedia.org", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "French Open 2025 - Wikipedia, wolna encyklopedia", "url": "https://pl.wikipedia.org/wiki/French_Open_2025", "content": "French Open 2025 - tenisowy turniej wielkoszlemowy, który odbył się w dniach 25 maja-8 czerwca, na ceglanych kortach Stade Roland Garros w Paryżu.Była to 124. edycja tych rozgrywek.Pula nagród wyniosła 56,4 mln e

### SQL

In [10]:
%pip install sqlalchemy requests

Note: you may need to restart the kernel to use updated packages.


Init db connection

In [11]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

Init LLM model

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Init  SQL toolkit

In [12]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fbadbbe3d40>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fbadbbe3d40>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fbadbbe3d40>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [13]:
from langgraph.prebuilt import create_react_agent

system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
"""
agent_executor = create_react_agent(llm, toolkit.get_tools(), prompt=system_message)

In [14]:
example_query = "Which country's customers spent the most?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_TfVt2RYOkkKPfebrESvc20CM)
 Call ID: call_TfVt2RYOkkKPfebrESvc20CM
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_vYXcz5uOstCake9jyqp0VARq)
 Call ID: call_vYXcz5uOstCake9jyqp0VARq
  Args:
    table_names: Customer
  sql_db_schema (call_7lr4Jt15r1FVjVSZLZvBirPL)
 Call ID: call_7lr4Jt15r1FVjVSZLZvBirPL
  Args:
    table_names: Invoice
  sql_db_schema (call_knzM4JyuB8Ce0J7TA3w5dLoQ)
 Call ID: call_knzM4JyuB8Ce0J7TA3w5dLoQ
  Args:
    table_names: InvoiceLine
========